# Dataset pre-processing

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import OrderedDict
from tqdm import tqdm, trange
import wp8.pre_processing.utils as utils
from wp8.pre_processing.template_match import TemplateMatch
from wp8.pre_processing.generate_labels import LabelsGenerator
from wp8.pre_processing.process_dataset import ProcessDataset

## Timestamps Extraction from video frames
Extracting timestamps and datalogger times from each frame of each video

In [ ]:
curr_video = './data/videos/Actor_1_Bed_PH CAM 8.mp4'
timestamp_roi = (47, 85, 811, 150)
tm = TemplateMatch(video_path=curr_video, element_type='timestamp', timestamp_roi=timestamp_roi)


In [ ]:
cap = cv2.VideoCapture(curr_video)
_, first_frame = cap.read()
cap.release()

tm.template_match(first_frame, test=True)

In [ ]:
timestamps = tm.extract_timestamps()

In [ ]:
timestamps[:10]

In [ ]:
timestamps_serie = pd.Series(timestamps)
timestamps_serie.to_excel('outputs/Actor_1_Bed_PH_timestamps.xlsx',sheet_name='Actor_1_Bed_PH')

## Labels file generation

In [ ]:
gl = LabelsGenerator(json_dir="../data/labels_json/")
gl.generate_labels()

## Features Extraction and Dataset excel creation

In [ ]:
videos_folder = '/Volumes/HDD ESTERNO Andrea/DATASET WP8'

feature_extractor = tf.keras.applications.InceptionV3(
    weights="imagenet",
    include_top=False,
    pooling="avg",
    input_shape=(224, 224, 3),
)

ds = ProcessDataset(videos_folder=videos_folder, feature_extractor=feature_extractor, preprocess_input=tf.keras.applications.inception_v3.preprocess_input)
ds.extract_frames()


In [ ]:
df = pd.read_excel("outputs/labels/labels.xlsx",
                   sheet_name="actor_1_bed_full_ph", index_col=0)

# df1

# augmented = pd.concat([df] * 7, ignore_index=True)
# augmented

In [ ]:
df.shape
df1 = pd.concat([df]*8)
df1.shape

In [ ]:
names = ["test"] * 4920

In [ ]:
df1["frame_name"] = pd.Series(names)


In [ ]:
features = np.load("/Users/andrea/Documents/Github/WP8_refactoring/wp8/outputs/dataset/features/Actor_1_Bed.npy")
dataset = pd.read_csv("/Users/andrea/Documents/Github/WP8_refactoring/wp8/outputs/dataset/dataset/full_dataset.csv")

print(f"dataset shape: {dataset.shape}\nfeatures shape: {features.shape}")
      
dataset["features"] = pd.Series(list(features))
dataset.head()
